In [8]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
import os
from IPython.display import display
import networkx as nx
from IPython.display import display_pdf, HTML
from pprint import pprint

In [2]:
df_article = pd.read_csv('./data/20221007_20231006_all.csv')

In [9]:
df_degree = pd.read_csv('./study_data/degree.csv')
df_flow = pd.read_csv('./study_data/all_cities_in_out_count.csv')
df_all = pd.read_csv('./study_data/score/城市信息+排名得分综合表.csv')

In [10]:
df_city_en = pd.read_csv('./study_data/sa340.csv')
df_city_en

,citycode,adcode,name,level,center,pro_name,pro_adcode,gzh,city_shortname,pro_shortname,en_name
0,315,130200,唐山市,city,"118.180149,39.63068",河北省,130000,唐山发布,唐山,河北,Tangshan
1,335,130300,秦皇岛市,city,"119.52022,39.888243",河北省,130000,秦皇岛发布,秦皇岛,河北,Qinhuangdao
2,314,130800,承德市,city,"117.962749,40.952942",河北省,130000,承德发布,承德,河北,Chengde
3,311,130100,石家庄市,city,"114.514976,38.042007",河北省,130000,石家庄发布,石家庄,河北,Shijiazhuang
4,316,131000,廊坊市,city,"116.683546,39.538304",河北省,130000,廊坊发布,廊坊,河北,Langfang
...,...,...,...,...,...,...,...,...,...,...,...
335,10,110000,北京市,province,"116.407387,39.904179",北京市,110000,北京发布,北京,北京,Beijing
336,22,120000,天津市,province,"117.201509,39.085318",天津市,120000,天津发布,天津,天津,Tianjin
337,21,310000,上海市,province,"121.473667,31.230525",上海市,310000,上海发布,上海,上海,Shanghai
338,23,500000,重庆市,province,"106.550483,29.563707",重庆市,500000,重庆发布,重庆,重庆,Chongqing


In [11]:
def show_pdf(city_full_name):
    pdf_path = './study_data/figs/city_intensity_cn_' + city_full_name + '.pdf'
    html_code = f'<iframe src="{pdf_path}" width="100%" height="600px"></iframe>'
    display(HTML(html_code))
    
# A被B报道的文章 
def show_who_in_other(A, B):
    for i in df_article[df_article.city_name==B]['text']:
        if A in i:
            print(i)
            
def show_detail(city_full_name):
    en_name = list(df_city_en[df_city_en.name==city_full_name]['en_name'])[0]
    r1 = df_flow[df_flow.cityname == city_full_name]
    citation_in = list(r1['in-count'])[0]
    citation_out = list(r1['out-count'])[0]
    # display(r1)
    
    r2 = df_degree[df_degree.name==city_full_name]
    in_degree = list(r2['in-degree'])[0]
    out_degree = list(r2['out-degree'])[0]
    sub_degree = in_degree - out_degree
    # display(r2)
    
    return [en_name, in_degree, out_degree, sub_degree]
    # more = df_all[df_all.name==city_full_name].iloc[0].to_dict()
    # pprint(more)
    # print('有效文章数：', len(df_article[df_article.city_name==city_full_name]))
    
def show_rec(name):
    r1 = df_all[df_all.name==name].to_dict(orient='records')[0]
    
    r2 = df_degree[df_degree.name==name]
    in_degree = list(r2['in-degree'])[0]
    out_degree = list(r2['out-degree'])[0]
    
    
    gzh = r1.get('gzh')
    citation_in = r1.get('in-count')
    in_degree_rank = r1.get('in-degree-rank')
    closeness_rank = r1.get('closeness-rank')
    pagerank_rank = r1.get('pagerank-rank')
    importance_score = r1.get('importance_score')
    ari = r1.get('ARI')
    
    articles = len(df_article[df_article.city_name==name])
    
    return [name, gzh, ari, citation_in, importance_score, 
            closeness_rank, in_degree_rank, pagerank_rank, in_degree, out_degree, articles]

## TOP10 & LAST10

In [12]:
top_10 = list(df_all.sort_values(by='ARI', ascending=False)[0:10]['name'])
las_10 = list(df_all.sort_values(by='ARI', ascending=False)[330:341]['name'])
cities = top_10 + las_10
print(len(list(set(cities))))

data = []
for i in cities:
    data.append(show_detail(i))
    
diff = pd.DataFrame(data, columns=['City Name', 'In Degree', 'Out Degree', 'Degree Difference'])
diff.to_csv('./study_data/table/top_10_last_10_degree_diff.csv', index=None)
diff

20


,City Name,In Degree,Out Degree,Degree Difference
0,Beijing,334,144,190
1,Shanghai,332,224,108
2,Guangzhou,323,112,211
3,Shenzhen,328,190,138
4,Hangzhou,326,181,145
5,Hong Kong,328,23,305
6,Chengdu,318,259,59
7,Chongqing,327,220,107
8,Tianjin,323,183,140
9,Wuhan,319,205,114
